In [1]:
import numpy as np

omaha_point = np.array((-95.995102, 41.257160)) 

## Pandas and scikit-learn's KDTree

In [2]:
import pandas as pd
from sklearn.neighbors import KDTree

In [5]:
url = 'http://www2.census.gov/geo/docs/maps-data/data/gazetteer/2016_Gazetteer/2016_Gaz_zcta_national.zip'
df_locations = pd.read_csv(url, dtype={'GEOID' : 'str'},sep='\t', usecols=[0,5,6])
df_locations.columns = df_locations.columns.str.strip()  #some column cleanup
print (len(df_locations))
df_locations.head()

33144


,GEOID,INTPTLAT,INTPTLONG
0,00601,18.180555,-66.749961
1,00602,18.361945,-67.175597
2,00603,18.455183,-67.119887
3,00606,18.158345,-66.932911
4,00610,18.295366,-67.125135


In [6]:
kdt = KDTree(df_locations[['INTPTLONG', 'INTPTLAT']])

In [7]:
omaha_point_kdt = np.expand_dims(omaha_point, axis=0)

nearest_point_index = kdt.query(omaha_point_kdt, k=1, return_distance=False)
print(df_locations.loc[nearest_point_index[0], 'GEOID'])

23609    68132
Name: GEOID, dtype: object


## GeoPandas and Shapely

In [8]:
import geopandas as gpd
from shapely.geometry import Point

In [10]:
#ftp://ftp2.census.gov/geo/tiger/TIGER2016/ZCTA5/tl_2016_us_zcta510.zip
gdf_locations = gpd.read_file('data/tl_2016_us_zcta510/tl_2016_us_zcta510.shp')
print(len(gdf_locations))
gdf_locations[['GEOID10', 'geometry']].head()

33144


,GEOID10,geometry
0,43451,"POLYGON ((-83.674464 41.331119, -83.6744449999..."
1,43452,"POLYGON ((-83.067745 41.537718, -83.067729 41...."
2,43456,"(POLYGON ((-82.8566 41.681222, -82.856831 41.6..."
3,43457,"POLYGON ((-83.467474 41.268186, -83.4676039999..."
4,43458,"POLYGON ((-83.222292 41.531025, -83.2222819999..."


In [11]:
omaha_point_shp = Point(omaha_point)

filter = gdf_locations['geometry'].contains(omaha_point_shp)
print(gdf_locations.loc[filter, 'GEOID10'])

24842    68132
Name: GEOID10, dtype: object


## Timing

In [12]:
%%timeit
nearest_point_index = kdt.query(omaha_point_kdt, k=1, return_distance=False)
df_locations.loc[nearest_point_index[0], 'GEOID']

563 µs ± 27.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
%%timeit
filter = gdf_locations['geometry'].contains(omaha_point_shp)
gdf_locations.loc[filter, 'GEOID10']

276 ms ± 17.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
